In [1]:
import mesa
from functional import seq
import random

from dataclasses import dataclass

In [2]:
from typing import Tuple

In [3]:
@dataclass
class Hotel():
    id: int
    cost: float
    max_capacity: int
    occupied: int = 0

    # stats
    new_rev: int = 0
    total_rev: int = 0

    def remaining_capacity(self) -> int:
        return self.occupied - self.occupied

In [4]:
@dataclass
class PointOfInterest():
    id: int
    cost: float
    max_capacity: int
    occupied: int = 0

    # stats
    visit: int = 0
    total_visit: int = 0

    def remaining_capacity(self) -> int:
        return self.occupied - self.occupied

In [69]:
class Tourist(mesa.Agent):
    def __init__(self, unique_id: int, model, loc: int, trip_length: int):
        super().__init__(unique_id, model)
        self.loc = loc
        self.hotel = None
        self.poi=None
        self.trip_length = trip_length
        self.remaining_life = trip_length

        self.satisfaction = 0 

    def step(self):
        if self.hotel is None and self.remaining_life >= 3:
            print("Looking for hotel")
            hotels = (
                seq(self.model.hotels)
                    .filter(lambda h: h.remaining_capacity() == 0)
                    .list()
            )

            if len(hotels) <= 0:
                print("NO HOTELS")
                # should delete ones self
                return;

            random.shuffle(hotels)

            self.hotel = hotels[0]
            self.hotel.occupied += 1
            return

        if self.poi is None:
            print("Looking for point of interest")
            no_capacity_poi=[p for p in self.model.poi if p.remaining_capacity()==0]


            if len(no_capacity_poi) > 0:
                print("NO Point of Interest")
                self.satisfaction -= 1
                print(f"self satisfaction: {self.satisfaction}")
                return self.satisfaction
            
            random.shuffle(self.model.poi)
            self.poi = self.model.poi[0]
            
            return
        
            


In [63]:
class Model(mesa.Model):
    def __init__(self, num_tourists, num_hotels, num_of_poi):
        self.hotels = []
        self.poi = []
        self.tourists = []

        self.num_tourists=num_tourists
        self.num_hotels=num_hotels
        self.num_of_poi=num_of_poi
        self.schedule=mesa.time.RandomActivation(self)

        # Stats
        
        
        # should be replaced with data from file
        for i in range (self.num_of_poi):
            self.hotels.append(Hotel(i, 200, 100))

        for i in range (self.num_hotels):
            self.poi.append(PointOfInterest(i, 200, 100))
            # self.schedule.add(hotel)

        for i in range(self.num_tourists):
            tourist = Tourist(i, self, 0, 10)
            self.tourists.append(tourist)
            self.schedule.add(tourist)

    def step(self):
        self.schedule.step()
        self.schedule.step()
        self.schedule.step()

    def hotel_stats(self) -> Tuple[float, float, float]:
        total_capacity: int = (seq(self.hotels)
            .map(lambda h: h.max_capacity)
            .fold_left(lambda acc, next: acc + next)
        )
        empty_rooms: int = (seq(self.hotels)
            .map(lambda h: h.remaining_capacity())
            .fold_left(lambda acc, next: acc + next)
        )
        occupied_rooms: int = (seq(self.hotels)
            .map(lambda h: h.occupied)
            .fold_left(lambda acc, next: acc + next)
        )
    
        return total_capacity

    def poi_stats(self) ->Tuple[float]:
        total_capacity_poi: int = (seq(self.poi)
            .map(lambda p: p.max_capacity)
            .fold_left(lambda acc, next: acc+next)
        )

        
        return total_capacity_poi

In [71]:
starter_model=Model(3, 3, 1)
for i in range(10):
    starter_model.step()

Looking for hotel
Looking for hotel
Looking for hotel
Looking for point of interest
NO Point of Interest
self satisfaction: -1
Looking for point of interest
NO Point of Interest
self satisfaction: -1
Looking for point of interest
NO Point of Interest
self satisfaction: -1
Looking for point of interest
NO Point of Interest
self satisfaction: -2
Looking for point of interest
NO Point of Interest
self satisfaction: -2
Looking for point of interest
NO Point of Interest
self satisfaction: -2
Looking for point of interest
NO Point of Interest
self satisfaction: -3
Looking for point of interest
NO Point of Interest
self satisfaction: -3
Looking for point of interest
NO Point of Interest
self satisfaction: -3
Looking for point of interest
NO Point of Interest
self satisfaction: -4
Looking for point of interest
NO Point of Interest
self satisfaction: -4
Looking for point of interest
NO Point of Interest
self satisfaction: -4
Looking for point of interest
NO Point of Interest
self satisfaction: 